In [25]:
import numpy as np
import scipy.interpolate as si
import pprint
import json
import time
from selenium.webdriver.common.action_chains import ActionChains
from random import randint
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
from datetime import date


options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
DRIVER_PATH = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(options = options, executable_path=DRIVER_PATH)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
  "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})

ua = UserAgent()
a = ua.random
user_agent = ua.random

driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent":user_agent})

print(user_agent)

Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.2 Safari/537.36


In [26]:
# Curve base:
points = [[0, 0], [0, 2], [2, 3], [4, 0], [6, 3], [8, 2], [8, 0]];
points = np.array(points)

x = points[:,0]
y = points[:,1]


t = range(len(points))
ipl_t = np.linspace(0.0, len(points) - 1, 100)

x_tup = si.splrep(t, x, k=3)
y_tup = si.splrep(t, y, k=3)

x_list = list(x_tup)
xl = x.tolist()
x_list[1] = xl + [0.0, 0.0, 0.0, 0.0]

y_list = list(y_tup)
yl = y.tolist()
y_list[1] = yl + [0.0, 0.0, 0.0, 0.0]

x_i = si.splev(ipl_t, x_list) # x interpolate values
y_i = si.splev(ipl_t, y_list) # y interpolate values

PARAMS

In [27]:
url = "https://www.inmuebles24.com/"
estado = "Edo. de mexico"
minprice = "1200000"
maxprice = "1700000"
tipo = "Otros"

In [28]:
driver.get(url)
wait = WebDriverWait(driver,100)

text_box = wait.until(ec.presence_of_element_located((By.CLASS_NAME, "rbt-input-main")))
text_box.send_keys(Keys.CONTROL + "a")
text_box.send_keys(Keys.DELETE)
text_box.send_keys(estado)
text_box.send_keys(Keys.ENTER)


dropdown_button = wait.until(ec.presence_of_element_located((By.CLASS_NAME, "button-more-filters")))

if dropdown_button.text == "Ver búsqueda avanzada":
    dropdown_button.click()


hidden_minprice_box = driver.find_element_by_id("price-min")
hidden_maxprice_box = driver.find_element_by_id("price-max")


minprice_box = wait.until(ec.visibility_of(hidden_minprice_box))
maxprice_box = wait.until(ec.visibility_of(hidden_maxprice_box))

minprice_box.send_keys(Keys.DELETE)
minprice_box.send_keys(Keys.CONTROL + "a")
minprice_box.send_keys(minprice)
maxprice_box.send_keys(Keys.CONTROL + "a")
maxprice_box.send_keys(Keys.DELETE)
maxprice_box.send_keys(maxprice)

dropdown_type = Select(driver.find_element_by_id("propertyType"))
dropdown_type.select_by_value(tipo)

button_go = driver.find_element_by_class_name("submit-filters")
button_go.click()
time.sleep(randint(6, 10))

In [29]:
def getTextbyClass(driver, strElement):
    text = 'NA'
    try:
        text = driver.find_element_by_class_name(strElement)
        text = text.text
    except:
        print(f'Could not find element {strElement}')
    return text

def getTextByID(driver, strElement):
    text = 'NA'
    try:
        text = driver.find_element_by_id(strElement)
        text = text.text
    except:
        print(f'Could not find element {strElement}')
    return text  
    

In [30]:
def process_card(listing, record):
    time.sleep(randint(3, 4))
    header = listing.find_element_by_class_name("posting-header")
    
    driver.execute_script("return arguments[0].scrollIntoView();", header)
    header.click()
    driver.switch_to.window(driver.window_handles[1])

    operation = wait.until(ec.presence_of_element_located((By.CLASS_NAME, "price-operation")))
    record["operation"] = operation.text
    price = wait.until(ec.presence_of_element_located((By.CLASS_NAME, "price-items")))
    record["price"] = price.text        
    
    record["expenses"] = getTextbyClass(driver,"block-expensas")
    record["title_header_sup"] = getTextbyClass(driver,"title-type-sup") 
    record["title_header_loc"] = getTextbyClass(driver,"title-location")
    
    try:
        gral_section = driver.find_element_by_class_name("article-section-description")
    except:
        print(f'Could not find element article-section-description')
    else:
        record["fecha_publicacion"] = getTextbyClass(gral_section, "section-date")    
        record["section_title"] = getTextbyClass(gral_section, "section-title") 
        record["descripcion"] = getTextByID(gral_section, "verDatosDescripcion")  
    
    attributes = gral_section.find_elements_by_class_name("icon-feature")
    
    for attribute in attributes:
        if "Terreno" in attribute.text:     
            record[f'terreno'] = attribute.text

        if "Construidos" in attribute.text:     
            record[f'Construidos'] = attribute.text
        
        if "Baños" in attribute.text:     
            record[f'Baños'] = attribute.text       
        
        if "Estacionamientos" in attribute.text:     
            record[f'Estacionamientos'] = attribute.text       
        
        if "Recámaras" in attribute.text:     
            record[f'Recámaras'] = attribute.text         
        
        if "Medio baño" in attribute.text:     
            record[f'Medio baño'] = attribute.text 
        
        if "Antigüedad" in attribute.text:     
            record[f'Antigüedad'] = attribute.text        
        
    additional_data = driver.find_elements_by_class_name("general-section")
    
    
    for data in additional_data:    
        if "Características generales" in data.text:     
            record[f'Gral'] = data.text

        if "Servicios" in data.text:     
            record[f'Serivicios'] = data.text          
            
        if "Amenidades" in data.text:     
            record[f'Amenidades'] = data.text  
            
        if "Exteriores" in data.text:     
            record[f'Exteriores'] = data.text   
            
        if "Ubicación" in data.text:     
            record[f'Ubicación'] = data.text              
            
    
    try:
        mapSrc = driver.find_element_by_id("static-map")
        mapSrc = mapSrc.get_attribute("src")
    except:
        mapSrc = "NA"
        
    record["mapSrc"] = mapSrc               
    record["link"] = driver.current_url
    record["date_processed"]  = date.today().strftime('%Y-%m-%d')
    
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [31]:
def processPage(pageNumber):
    action =  ActionChains(driver)
    list_container = wait.until(ec.presence_of_element_located((By.CLASS_NAME, "list-card-container")))
    listing = list_container.find_elements_by_class_name("posting-card")

    recordlist = []
    counter = 1

    for element in listing:
        
        record = {}
        record["data-id"] = element.get_attribute("data-id")
        record["type"] = element.get_attribute("data-posting-type")      
        
        
        if record["type"] != "DEVELOPMENT":
        
            action.move_to_element(element);
            action.perform();   
            start_time = time.time()
            randomWait = (randint(2,5))
    
            for mouse_x, mouse_y in zip(x_i, y_i):
                if start_time + randomWait > time.time():
                    action.move_by_offset(mouse_x,mouse_y)
                    action.perform()
                else:
                    break
            
            recordlist.append(record)
            process_card(element, record)    
            print(f'ELEMENT {counter} PROCESSED SUCESSFULLY')
            print(f'---------------------------------------')
            
        else:
            print(f'ELEMENT {counter} SKIPPING DEVELOPMENT')
            print(f'---------------------------------------')
        counter = counter + 1
        

    outfile = (f'data_{estado}_{pageNumber}.json')
    with open(outfile, 'w') as out:
        json.dump(recordlist, out,  sort_keys=True, indent=4)

In [32]:
pageNumber = 1

while True:
    processPage(pageNumber)
    pageNumber = pageNumber + 1
    
    try:
        next_page = driver.find_element_by_class_name("pag-go-next")
        next_page.click()
        time.sleep(randint(6, 10))
    except Exception as e:
        print("All pages were processed")
        break
    

ELEMENT 1 SKIPPING DEVELOPMENT
---------------------------------------
ELEMENT 2 SKIPPING DEVELOPMENT
---------------------------------------
Could not find element block-expensas
ELEMENT 3 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element block-expensas
ELEMENT 4 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element block-expensas
ELEMENT 5 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element block-expensas
ELEMENT 6 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element block-expensas
ELEMENT 7 PROCESSED SUCESSFULLY
---------------------------------------
Could not find element block-expensas
ELEMENT 8 PROCESSED SUCESSFULLY
---------------------------------------
ELEMENT 9 PROCESSED SUCESSFULLY
---------------------------------------


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=84.0.4147.89)
